<a href="https://colab.research.google.com/github/vifirsanova/empi/blob/main/fine-tuning/load_model_to_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading fine-tuned model to the HuggingFace hub

## Load libraries

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.9 MB/s eta 0:00:00


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper

In [8]:
!cp -r "/content/drive/MyDrive/model/" "/content/model.pth" # this is my fine-tuned model weights

In [9]:
import torch

from transformers import BertForQuestionAnswering
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast

## Load model

In [22]:
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased") # base model

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [23]:
model.load_state_dict(torch.load('/content/model.pth/model.pth', map_location=torch.device('cpu'))) # apply my model loaded from Drive

<All keys matched successfully>

In [24]:
model.push_to_hub("ru-asd-qa-bert") # push it to HF repo

CommitInfo(commit_url='https://huggingface.co/missvector/ru-asd-qa-bert/commit/790db0bee8347b19f43fd7c6d6c6b86f9267a025', commit_message='Upload BertForQuestionAnswering', commit_description='', oid='790db0bee8347b19f43fd7c6d6c6b86f9267a025', pr_url=None, pr_revision=None, pr_num=None)

## Check the model

In [25]:
model = BertForQuestionAnswering.from_pretrained("missvector/ru-asd-qa-bert") # load model from my repo

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [28]:
# make any query to check that MRC works on the model

question, text = 'Вопрос?', 'Здесь есть ответ на вопрос. Ты нашел его?'

input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"

tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
input_ids = tokenizer.encode(input_text)

start_scores, end_scores = model(torch.tensor([input_ids])).values()

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
ans = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

In [29]:
ans # get the answer

'Здесь'